## 按行点击即可进行自动炼丹
## 等完全运行完后再点下一行
## 不要跨行运行或着连点多行

# Clone Github Repo

In [ ]:
!git clone https://github.com/AlexandaJerry/vits-mandarin-biaobei
%cd vits-mandarin-biaobei

In [ ]:
!/opt/bin/nvidia-smi

In [ ]:
!pip install -r requirements.txt
!sudo apt-get install espeak -y

# Unpack dataset

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1BhRQBNcoEq4tprikwEy0RSOWflTrFwYD' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1BhRQBNcoEq4tprikwEy0RSOWflTrFwYD" -O biaobei.rar && rm -rf /tmp/cookies.txt

In [ ]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
%cd vits-mandarin-biaobei
%mkdir biaobei
Archive('biaobei.rar').extractall('/content/vits-mandarin-biaobei/')

In [ ]:
!sudo apt-get install sox

In [ ]:
%cd /content/vits-mandarin-biaobei/biaobei

In [8]:
%%bash
for x in ./*.wav
do 
  b=${x##*/}
  sox $b -r 22050 tmp_$b
  rm -rf $b
  mv tmp_$b $b
done

# Alignment and Text Clean

In [ ]:
import os
path = "/content/vits-mandarin-biaobei"
os.chdir(path)
print(os.getcwd())

In [ ]:
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..

In [ ]:
import os
path = "/content/vits-mandarin-biaobei"
os.chdir(path)
print(os.getcwd())

In [ ]:
!python preprocess.py --text_index 1 --text_cleaners chinese_cleaners1 --filelists /content/vits-mandarin-biaobei/filelists/train_filelist.txt /content/vits-mandarin-biaobei/filelists/val_filelist.txt

# Train

In [13]:
#Tesla-P100-16G的GPU每个epoch大概需要5分钟左右
#每经过约3个epoch会存储新的checkpoint到/content/vits-mandarin-biaobei/logs/biaobei_base
#如果提示显存不足的话，需要去/content/vits-mandarin-biaobei/configs/biaobei_base.json
#把里面的batch_size从16降到8，然后重新运行该代码框
!python train.py -c configs/biaobei_base.json -m biaobei_base

---


---


#Inference:
#上个板块训练差不多后在菜单栏"代码执行程序"的下拉菜单选择"中断执行"
#然后在上方菜单栏"代码执行程序"的下拉菜单选择"重新启动代码执行程序"
#然后从下面这个代码框开始，按照顺序依次运行，最后即可得到合成的语音
#流程务必遵循：中断执行→重新启动代码执行程序→运行后续代码框

In [ ]:
import os
path = "/content/vits-mandarin-biaobei"
os.chdir(path)
print(os.getcwd())

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [2]:
hps = utils.get_hparams_from_file("/content/vits-mandarin-biaobei/configs/biaobei_base.json")

#下面这个"/content/vits-mandarin-biaobei/logs/biaobei_base/G_10000.pth"需要去看下
#VITS-MANDARIN-BIAOBEI文件夹下的log文件夹里的G_?????.pth后缀的数字最大是多少

In [3]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("/content/vits-mandarin-biaobei/logs/biaobei_base/G_?????.pth", net_g, None)

In [ ]:
stn_tst = get_text("别在这里发癫。", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))